# Importing Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)

import pandas as pd
import os

# file_path = '/content/drive/MyDrive/NNDL_HW2/HW2_Dataset/Train/'
# file1_path = '/content/drive/MyDrive/NNDL_HW2/HW2_Dataset/Train/Cats'
# file2_path = '/content/drive/MyDrive/NNDL_HW2/HW2_Dataset/Train/Dogs'

train_path = '/content/drive/MyDrive/NNDL_HW2_After_Augment/Train/'
train_path = '/content/drive/MyDrive/NNDL_HW2_After_Augment/'

all_test_path = '/content/drive/MyDrive/NNDL_HW2_Before_Augmentation/HW2_Dataset/Test/'

# test_path = '/content/drive/MyDrive/NNDL_HW2_Before_Augment/HW2_Dataset/Test/'
test_path = '/content/drive/MyDrive/NNDL_HW2_Before_Augmentation/HW2_Dataset/Test/'

files1 = [f for f in os.listdir(train_path) if os.path.isfile(os.path.join(train_path, f))]
files2 = [f for f in os.listdir(test_path) if os.path.isfile(os.path.join(test_path, f))]

test_files = [f for f in os.listdir(all_test_path) if os.path.isfile(os.path.join(all_test_path, f))]

print(f"Number of files in {train_path} After Augmentation : {len(files1)}")
print(f"Number of files in {test_path} Before Augmentation : {len(files2)}")
print('-----------------------------------------------')
print(f'Number of files in {test_path} : {len(test_files)}')

# df.head()

Mounted at /content/drive
Number of files in /content/drive/MyDrive/NNDL_HW2_After_Augment/ After Augmentation : 0
Number of files in /content/drive/MyDrive/NNDL_HW2_Before_Augmentation/HW2_Dataset/Test/ Before Augmentation : 0
-----------------------------------------------
Number of files in /content/drive/MyDrive/NNDL_HW2_Before_Augmentation/HW2_Dataset/Test/ : 0


# ResNet with keras

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define hyperparameters
initial_lr = 0.001
lr_decay_rate = 0.1
momentum = 0.9
batch_size = 32
epochs = 10

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Replace the FC layers with new ones for binary classification (dog vs cat)
x = Flatten()(base_model.output)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Freeze all layers except the new FC layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
opt = SGD(learning_rate=initial_lr, momentum=momentum)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation for training images
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)  # Splitting the dataset into training and validation

all_train_path = '/content/drive/MyDrive/NNDL_HW2_After_Augment/'
train_path = '/content/drive/MyDrive/NNDL_HW2_After_Augment/Train'

train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size=(224, 224),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    subset='training')  # Training set

validation_generator = train_datagen.flow_from_directory(train_path,
                                                         target_size=(224, 224),
                                                         batch_size=batch_size,
                                                         class_mode='categorical',
                                                         subset='validation')  # Validation set

# Train the model with training and validation data
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Unfreeze the last CONV block
for layer in model.layers[:15]:
    layer.trainable = False

# Recompile the model to apply the changes
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

all_test_path = '/content/drive/MyDrive/NNDL_HW2_Before_Augmentation/HW2_Dataset/Test/'

# Test data generator for evaluation
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(all_test_path ,
                                                  target_size=(224, 224),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

# Evaluate the model on the test set
model.evaluate(test_generator)

94765736/94765736 [==============================] - 0s 0us/step
Found 1685 images belonging to 2 classes.
Found 421 images belonging to 2 classes.
Epoch 1/10
53/53 [==============================] - 571s 11s/step - loss: 58.4494 - accuracy: 0.5187 - val_loss: 48.3054 - val_accuracy: 0.4988
Epoch 2/10
53/53 [==============================] - 480s 9s/step - loss: 28.6028 - accuracy: 0.5513 - val_loss: 5.2000 - val_accuracy: 0.6936
Epoch 3/10
53/53 [==============================] - 472s 9s/step - loss: 11.4091 - accuracy: 0.5638 - val_loss: 11.1124 - val_accuracy: 0.5416
Epoch 4/10
53/53 [==============================] - 406s 8s/step - loss: 21.6067 - accuracy: 0.5418 - val_loss: 5.9293 - val_accuracy: 0.6746
Epoch 5/10
53/53 [==============================] - 407s 8s/step - loss: 12.2736 - accuracy: 0.5804 - val_loss: 3.6995 - val_accuracy: 0.6936
Epoch 6/10
53/53 [==============================] - 468s 9s/step - loss: 15.9047 - accuracy: 0.5537 - val_loss: 14.8687 - val_accuracy: 0.5

[20.413557052612305, 0.5099999904632568]

# ResNet in PyTorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets

# Define hyperparameters
initial_lr = 0.001
momentum = 0.9
batch_size = 32
epochs = 10

# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Replace the FC layer for binary classification

# Freeze all layers except the new FC layer
for param in model.parameters():
    param.requires_grad = False
model.fc.requires_grad = True

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=initial_lr, momentum=momentum)

# Data augmentation and loading
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = '/content/drive/MyDrive/NNDL_HW2_After_Augment/Train/'
image_datasets = {x: datasets.ImageFolder(data_dir + x, data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'test']}

# Train the model
for epoch in range(epochs):
    for inputs, labels in dataloaders['train']:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print('Test Accuracy: {:.2f}%'.format(100 * accuracy))

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/NNDL_HW2_After_Augment/Train/train'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Define paths
all_train_path = '/content/drive/MyDrive/NNDL_HW2_After_Augment/Train/'
all_test_path = '/content/drive/MyDrive/NNDL_HW2_Before_Augmentation/HW2_Dataset/Test/'

# Load pre-trained ResNet50 model
model = models.resnet50(pretrained=True)

# Freeze all CONV layers
for param in model.parameters():
    param.requires_grad = False

# Replace FC layers with new FC layer for Cats and Dogs classification
num_classes = 2
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1, momentum=0.9)

# Load train and test datasets
train_dataset = ImageFolder(all_train_path, transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
]))
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

test_dataset = ImageFolder(all_test_path, transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
]))
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Train the network
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Unfreeze the last CONV block
for param in model.layer4.parameters():
    param.requires_grad = True

# Fine-tune the network
num_epochs_finetune = 50
for epoch in range(num_epochs_finetune):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs_finetune}, Loss: {loss.item()} , Accuracy : {model.accuracy()}')

# Evaluate on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Accuracy on test set: {100 * accuracy}%')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/50, Loss: 0.0
Epoch 2/50, Loss: 0.0
Epoch 3/50, Loss: 0.0
Epoch 4/50, Loss: 0.0
Epoch 5/50, Loss: 14.829227447509766
Epoch 6/50, Loss: 1.4901160305669237e-08
Epoch 7/50, Loss: 0.0
Epoch 8/50, Loss: 0.0
Epoch 9/50, Loss: 0.0
Epoch 10/50, Loss: 0.10322771221399307
